# Final Report #

In [3]:
#load in packages
library(tidymodels)
library(tidyverse)

#read in datasets
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ recipes      1.1.0
✔ dials        1.3.0     ✔ rsample      1.2.1
✔ dplyr        1.1.4     ✔ tibble       3.2.1
✔ ggplot2      3.5.1     ✔ tidyr        1.3.1
✔ infer        1.0.7     ✔ tune         1.1.2
✔ modeldata    1.4.0     ✔ workflows    1.1.4
✔ parsnip      1.2.1     ✔ workflowsets 1.0.1
✔ purrr        1.0.2     ✔ yardstick    1.3.1

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Use suppressPackageStartupMessages() to eliminate package startup messages

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ lubridate 1.9.3     ✔ stringr   1.5.1
── Conflicts ────────────────────────────────────────── tidyverse_co

## Introduction 
### Background Information: 

### Scientific Question: 



In [6]:
summary(players)
nrow(players)

  experience        subscribe       hashedEmail         played_hours    
 Length:196         Mode :logical   Length:196         Min.   :  0.000  
 Class :character   FALSE:52        Class :character   1st Qu.:  0.000  
 Mode  :character   TRUE :144       Mode  :character   Median :  0.100  
                                                       Mean   :  5.846  
                                                       3rd Qu.:  0.600  
                                                       Max.   :223.100  
                                                                        
     name              gender               Age       
 Length:196         Length:196         Min.   : 9.00  
 Class :character   Class :character   1st Qu.:17.00  
 Mode  :character   Mode  :character   Median :19.00  
                                       Mean   :21.14  
                                       3rd Qu.:22.75  
                                       Max.   :58.00  
                               

[1] 196

### Dataset Used: 
`players.csv` Dataset:
- 196 observations
- 7 variables
  
|#| Variable Name | Min | Mean | Median | Max |
|:--------:|:--------|:--------|:--------|:--------:|:--------:|
|1| `played_hours`  | 0.00 | 5.845 | 0.10 | 223.10|
|2|`Age`| 9.00| 21.14| 19.00 | 58.00|

#### Issues:
- few NA values are present
- `Age` should be an integer instead of double
-  Some `played_hours` values are 0.0 which could indicate missing data.

### How the Data was Collected:
The data was collected by a UBC Computer Science research group led by Frank Wood, recording players’ actions on a Minecraft server during gameplay.